In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from pyspark.sql import SparkSession
from delta import *

import pyspark.sql.functions as F
import os
import time

## Spark Session

In [2]:
# Create SparkSession with Delta Lake support
# Prepare the Spark builder
builder = SparkSession.builder.appName("project_3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.executor.memory", "6gb") \
    .config("spark.driver.memory", "12g") 

spark = configure_spark_with_delta_pip(builder,extra_packages=["graphframes:graphframes:0.8.4-spark3.5-s_2.12"]).getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", spark._sc.defaultParallelism)

spark.conf.set("spark.sql.repl.eagerEval.enabled",True) # OK for exploration, not great for performance
spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)

import graphframes as gf


## Dataset read

In [3]:
flight_df = spark.read.csv("data/2009.csv", header=True, inferSchema=True)
display(flight_df)

FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,1158.0,8.0,1202,1206.0,4.0,0.0,NULL,0.0,62.0,68.0,42.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,1620.0,4.0,1632,1624.0,-8.0,0.0,NULL,0.0,82.0,75.0,43.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,1155.0,6.0,1210,1201.0,-9.0,0.0,NULL,0.0,70.0,62.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,1336.0,9.0,1357,1345.0,-12.0,0.0,NULL,0.0,77.0,56.0,37.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,1809.0,13.0,1900,1822.0,-38.0,0.0,NULL,0.0,105.0,77.0,40.0,213.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,ATL,EWR,1915,1913.0,-2.0,19.0,1932.0,2108.0,15.0,2142,2123.0,-19.0,0.0,NULL,0.0,147.0,130.0,96.0,745.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1212,CLE,ATL,1645,1637.0,-8.0,12.0,1649.0,1820.0,5.0,1842,1825.0,-17.0,0.0,NULL,0.0,117.0,108.0,91.0,554.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1214,DCA,EWR,1915,1908.0,-7.0,9.0,1917.0,1953.0,34.0,2035,2027.0,-8.0,0.0,NULL,0.0,80.0,79.0,36.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1215,EWR,DCA,1715,1710.0,-5.0,28.0,1738.0,1819.0,4.0,1838,1823.0,-15.0,0.0,NULL,0.0,83.0,73.0,41.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL
2009-01-01,XE,1217,EWR,DCA,1300,1255.0,-5.0,15.0,1310.0,1349.0,7.0,1408,1356.0,-12.0,0.0,NULL,0.0,68.0,61.0,39.0,199.0,NULL,NULL,NULL,NULL,NULL,NULL


In [4]:
flight_df.printSchema()

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: doub

## Filtering data

### Edges

In [5]:
flight_edgs_df = (flight_df
    .filter(
        F.column("ORIGIN").isNotNull() & F.column("DEST").isNotNull()
    )
    .select(
        F.column("ORIGIN").alias("src"), 
        F.column("DEST").alias("dst"), 
        F.column("FL_DATE"), 
        F.column("CANCELLED"), 
        F.column("ARR_TIME"), 
        F.column("DISTANCE"), 
    )
)
display(flight_edgs_df)

src,dst,FL_DATE,CANCELLED,ARR_TIME,DISTANCE
DCA,EWR,2009-01-01,0.0,1206.0,199.0
EWR,IAD,2009-01-01,0.0,1624.0,213.0
EWR,DCA,2009-01-01,0.0,1201.0,199.0
DCA,EWR,2009-01-01,0.0,1345.0,199.0
IAD,EWR,2009-01-01,0.0,1822.0,213.0
ATL,EWR,2009-01-01,0.0,2123.0,745.0
CLE,ATL,2009-01-01,0.0,1825.0,554.0
DCA,EWR,2009-01-01,0.0,2027.0,199.0
EWR,DCA,2009-01-01,0.0,1823.0,199.0
EWR,DCA,2009-01-01,0.0,1356.0,199.0


### Vertex

In [6]:
flight_vertex_df = (flight_df
    .filter(
        F.column("ORIGIN").isNotNull() & F.column("DEST").isNotNull()
    )
    .select(
        F.column("ORIGIN").alias("id"),
    )
    .union(flight_df
        .select(
            F.column("DEST").alias("id"),
        )
    )
    .distinct()
)
display(flight_vertex_df)

id
JAN
JAX
ABQ
ORF
TUL
STL
GPT
SYR
CID
FAT


In [7]:
flight_graph = gf.GraphFrame(flight_vertex_df, flight_edgs_df)
flight_vertex_df.cache()
flight_edgs_df.cache()

display(flight_graph)

GraphFrame(v:[id: string], e:[src: string, dst: string ... 4 more fields])

In [8]:
display(flight_graph.vertices)

id
JAN
JAX
ABQ
ORF
TUL
STL
GPT
SYR
CID
FAT


In [9]:
display(flight_graph.edges)

src,dst,FL_DATE,CANCELLED,ARR_TIME,DISTANCE
DCA,EWR,2009-01-01,0.0,1206.0,199.0
EWR,IAD,2009-01-01,0.0,1624.0,213.0
EWR,DCA,2009-01-01,0.0,1201.0,199.0
DCA,EWR,2009-01-01,0.0,1345.0,199.0
IAD,EWR,2009-01-01,0.0,1822.0,213.0
ATL,EWR,2009-01-01,0.0,2123.0,745.0
CLE,ATL,2009-01-01,0.0,1825.0,554.0
DCA,EWR,2009-01-01,0.0,2027.0,199.0
EWR,DCA,2009-01-01,0.0,1823.0,199.0
EWR,DCA,2009-01-01,0.0,1356.0,199.0


# QUERY 1

## Expected results

In [11]:
expected_degrees_df = flight_graph.degrees
expected_inDegrees_df = flight_graph.inDegrees
expected_outDegrees_df = flight_graph.outDegrees
expected_triangles_count_df = flight_graph.triangleCount()

In [12]:
# Taken from https://stackoverflow.com/questions/31197353/dataframe-equality-in-apache-spark
def are_dfs_equal(df1, df2):
    if df1.schema != df2.schema:
        print("schema is not the same")
        return False
    if df1.collect() != df2.collect():
        print("data is not the same")
        return False
    return True

## inDegrees

In [14]:
in_edges_df = (flight_graph
    .find("()-[edge]->(a)")
    .groupBy("a.id")
    .count()
    .withColumn("inDegree", F.col("count").cast(IntegerType()))
    .select(F.col("id"), F.col("inDegree"))

)

display(in_edges_df)

id,inDegree
JAX,28813
ABQ,35577
ORF,15245
JAN,12528
TUL,20731
GPT,6588
STL,58691
SYR,9330
RKS,1764
PWM,6510


In [16]:
are_dfs_equal(in_edges_df, expected_inDegrees_df)

True

## outDegrees

In [17]:
outgoing_edges_df = (flight_graph
    .find("(a)-[edge]->()")
    .groupBy("a.id")
    .count()
    .withColumn("outDegree", F.col("count").cast(IntegerType()))
    .select(F.col("id"), F.col("outDegree"))

)

display(outgoing_edges_df)

id,outDegree
JAN,12530
JAX,28810
ABQ,35582
ORF,15238
TUL,20735
STL,58702
GPT,6588
SYR,9336
CID,9049
FAT,12319


In [19]:
are_dfs_equal(outgoing_edges_df, expected_outDegrees_df)

True

## Degrees

In [20]:
degrees_df = (outgoing_edges_df
    .join(in_edges_df, "id")
    .withColumn("degree", F.col("outDegree")+ F.col("inDegree"))
    .select(F.col("id"), F.col("degree"))
)
display(degrees_df)

id,degree
JAN,25058
JAX,57623
ABQ,71159
ORF,30483
TUL,41466
STL,117393
GPT,13176
SYR,18666
CID,18096
FAT,24636


In [21]:
expected_degrees_df.count(), degrees_df.count()

(296, 296)

In [22]:
letsCheck_df = (expected_degrees_df
    .withColumnRenamed("degree", "expected_degree")
    .join(degrees_df, "id")
    .withColumn("diff", F.col("expected_degree") -  F.col("degree"))
    .filter(F.col("diff")!= 0)
    .select(F.col("id"), F.col("diff"), F.col("expected_degree"), F.col("degree"))
)
display(letsCheck_df)

id,diff,expected_degree,degree


## Triangles count

## mofits -> It takes more than an hour to compute

In [ ]:
triangles_count_df = (flight_graph
    .find("(a)-[ab]->(b);(b)-[bc]->(c);(c)-[ca]->(a)")
    .where("a.id < b.id AND b.id < c.id")
    .groupBy("a.id")
    .count()
    .select(F.col("id"), F.col("count"))

)

display(triangles_count_df)

## Joins

In [12]:
undirected_edges = (flight_edgs_df
                    .select(
                        F.when(F.col("src") < F.col("dst"), F.col("src")).otherwise(F.col("dst")).alias("v1"),
                        F.when(F.col("src") < F.col("dst"), F.col("dst")).otherwise(F.col("src")).alias("v2")
                    )
                    .dropDuplicates())

In [13]:
# Alias the undirected edges for the self-join.
e1 = undirected_edges.alias("e1")
e2 = undirected_edges.alias("e2")
e3 = undirected_edges.alias("e3")

# Join e1 and e2 to form potential chains: v1 - v2 - v3
triangle_candidates = e1.join(e2, F.col("e1.v2") == F.col("e2.v1"))

# Now join with e3 to check if an edge exists between v1 and v3 to close the triangle.
triangles = triangle_candidates.join(
    e3,
    (F.col("e1.v1") == F.col("e3.v1")) & (F.col("e2.v2") == F.col("e3.v2"))
)

# Filter triangles to remove duplicates (v1 < v2 < v3)
triangles = triangles.filter(
    (F.col("e1.v1") < F.col("e1.v2")) &
    (F.col("e1.v2") < F.col("e2.v2"))
)

In [15]:
# Compute per-vertex triangle counts: each triangle involves three airports.
# Union the vertices from each triangle and then count how many times each airport appears.
triangle_vertices = (triangles.select(F.col("e1.v1").alias("id"))
                     .union(triangles.select(F.col("e1.v2").alias("id")))
                     .union(triangles.select(F.col("e2.v2").alias("id"))))

vertex_triangle_counts = triangle_vertices.groupBy("id").agg(F.count("*").alias("triangle_count"))
display(vertex_triangle_counts)

id,triangle_count
HOU,262
ABQ,311
CID,36
LAX,1020
SEA,728
SAN,565
FAT,31
GPT,21
JAN,36
JAX,342


# Query 2. Total Number of Triangles in the Graph

In [16]:
triangle_total_count = triangles.count()
print("Total number of triangles (overall):", triangle_total_count)

Total number of triangles (overall): 16015


# Query 3. Total Number of Triangles in the Graph

In [20]:
# Initialize each vertex with a rank (centrality) of 1.0.
ranks = flight_vertex_df.select("id").withColumn("rank", F.lit(1.0))

# Set parameters for the power iteration.
max_iter = 10
epsilon = 1e-4

for i in range(max_iter):
    # Each vertex's rank is distributed to its destination neighbors.
    contribs = flight_edgs_df.join(ranks, flight_edgs_df.src == ranks.id) \
                    .groupBy("dst") \
                    .agg(F.sum("rank").alias("contrib"))
    
    # Create new ranks by joining these contributions to the vertices.
    new_ranks = flight_vertex_df.join(contribs, flight_vertex_df.id == contribs.dst, "left") \
                        .select(flight_vertex_df.id,
                                F.coalesce(F.col("contrib"), F.lit(0.0)).alias("new_rank"))
    
    # Normalize the new ranks so that the sum of ranks equals 1.
    total_rank = new_ranks.agg(F.sum("new_rank")).first()[0]
    new_ranks = new_ranks.withColumn("new_rank", F.col("new_rank") / total_rank)
    
    # Check convergence.
    joined = ranks.join(new_ranks, "id")
    diff = joined.withColumn("diff", F.abs(F.col("rank") - F.col("new_rank")))
    max_diff_value = diff.agg(F.max("diff")).first()[0]
    print("Iteration", i, "max change:", max_diff_value)
    
    if max_diff_value < epsilon:
        break
    
    # Update the ranks for the next iteration.
    ranks = new_ranks.withColumnRenamed("new_rank", "rank")

# Display the computed eigenvector centrality scores.
print("Eigenvector Centrality Scores:")
ranks.orderBy(F.col("rank").desc()).show(10)

Iteration 0 max change: 0.9999995333889741
Iteration 1 max change: 0.033631523333202455
Iteration 2 max change: 0.008680665250810814
Iteration 3 max change: 0.0031526869477974603
Iteration 4 max change: 0.0009518548397060012
Iteration 5 max change: 0.00034457981182840075
Iteration 6 max change: 0.00010000117639264589
Iteration 7 max change: 3.908776374717676e-05
Eigenvector Centrality Scores:
+---+--------------------+
| id|                rank|
+---+--------------------+
|ATL| 0.03753374418630729|
|ORD| 0.03420041245905938|
|DFW|0.029195453879788374|
|DEN| 0.02846066233421956|
|LAX|0.027571457107331145|
|PHX| 0.02469136002748945|
|LAS|0.023324443940525116|
|IAH|0.020739376543623195|
|SFO| 0.01988898809777415|
|BOS| 0.01879929142273175|
+---+--------------------+
only showing top 10 rows



# Query 4. Implement the PageRank algorithm natively on Spark using Graphframes

In [21]:
damping = 0.85
max_iter = 10
epsilon = 1e-4

# Pre-compute out-degrees for normalization.
out_degrees = flight_edgs_df.groupBy("src").agg(F.count("dst").alias("out_degree"))

# Initialize PageRank values.
pranks = flight_vertex_df.select("id").withColumn("rank", F.lit(1.0))

for i in range(max_iter):
    # Calculate contributions from each vertex to its neighbors.
    edge_contribs = (flight_edgs_df
                     .join(pranks, flight_edgs_df.src == pranks.id)
                     .join(out_degrees, flight_edgs_df.src == out_degrees.src)
                     .select(flight_edgs_df.dst,
                             (F.col("rank") / F.col("out_degree")).alias("contrib")))
    
    # Sum contributions by destination vertex.
    summed_contribs = edge_contribs.groupBy("dst") \
                                   .agg(F.sum("contrib").alias("total_contrib"))
    
    # Apply the PageRank update rule.
    new_pranks = (flight_vertex_df
                  .join(summed_contribs, flight_vertex_df.id == summed_contribs.dst, "left")
                  .select(flight_vertex_df.id,
                          ((1 - damping) + damping * F.coalesce(F.col("total_contrib"), F.lit(0.0)))
                           .alias("new_rank")))
    
    # Check convergence.
    joined_ranks = pranks.join(new_pranks, "id")
    diff = joined_ranks.withColumn("diff", F.abs(F.col("rank") - F.col("new_rank")))
    max_diff_value = diff.agg(F.max("diff")).first()[0]
    print("Iteration", i, "max change:", max_diff_value)
    
    if max_diff_value < epsilon:
        break
        
    pranks = new_pranks.withColumnRenamed("new_rank", "rank")

print("PageRank Scores:")
pranks.orderBy(F.col("rank").desc()).show(10)

Iteration 0 max change: 37.87100076226611
Iteration 1 max change: 26.09260776330634
Iteration 2 max change: 9.466411593977632
Iteration 3 max change: 4.944223546568683
Iteration 4 max change: 2.2046881069997575
Iteration 5 max change: 0.9986999029300101
Iteration 6 max change: 0.5108192893470793
Iteration 7 max change: 0.19801791533137703
Iteration 8 max change: 0.12155779308209702
Iteration 9 max change: 0.035918004299670514
PageRank Scores:
+---+------------------+
| id|              rank|
+---+------------------+
|ATL|18.905010413236592|
|ORD| 12.92711042270102|
|DFW|11.735613068886858|
|DEN| 9.998483451765958|
|LAX| 7.726101869874977|
|IAH| 7.159305038125105|
|PHX|7.0653715625994264|
|SLC| 7.038754134167003|
|DTW| 7.019968536748528|
|SFO| 5.904248175402836|
+---+------------------+
only showing top 10 rows



# Query 5. Finding the Group of the Most Connected Airports

In [22]:
# Create an Undirected Edges DataFrame 
# For connectivity, treat the graph as undirected by unioning edges with their reverse.
undirected_edges = (flight_edgs_df.select("src", "dst")
                    .union(flight_edgs_df.select(F.col("dst").alias("src"), F.col("src").alias("dst")))
                    .dropDuplicates())

print("Undirected edges sample:")
undirected_edges.show(5)

Undirected edges sample:
+---+---+
|src|dst|
+---+---+
|IAH|MKE|
|CLE|BOS|
|GRR|CLE|
|MKE|IAH|
|OMA|IAH|
+---+---+
only showing top 5 rows



In [23]:
# Initialize Vertices with Component Labels
# Initially, each vertex is its own component.
vertices_df = flight_vertex_df.withColumn("component", F.col("id"))

print("Initial vertices with component labels:")
vertices_df.show(5)


Initial vertices with component labels:
+---+---------+
| id|component|
+---+---------+
|JAN|      JAN|
|JAX|      JAX|
|ABQ|      ABQ|
|ORF|      ORF|
|TUL|      TUL|
+---+---------+
only showing top 5 rows



In [24]:
# Iterative Union-Find (Label Propagation)
max_iter = 10  
iteration = 0
converged = False

while not converged and iteration < max_iter:
    # For each vertex, get the minimum component label among its neighbors.
    # Join undirected_edges with vertices_df so we can retrieve neighbor's current component.
    neighbor_components = undirected_edges.join(
        vertices_df, undirected_edges.dst == vertices_df.id, "left"
    ).groupBy("src") \
     .agg(F.min("component").alias("min_neighbor_component"))
    
    # Now, for each vertex, update its component to the minimum of its current value and the minimum found among neighbors.
    updated_vertices = vertices_df.join(
        neighbor_components, vertices_df.id == neighbor_components.src, "left"
    ).withColumn(
        "new_component",
        F.when(F.col("min_neighbor_component").isNotNull(),
               F.least(F.col("component"), F.col("min_neighbor_component"))
              ).otherwise(F.col("component"))
    ).select("id", "new_component")
    
    # Check for convergence: count how many vertices change their component label.
    diff_count = updated_vertices.filter(F.col("new_component") != vertices_df.component).count()
    print("Iteration", iteration, "number of component updates:", diff_count)
    
    if diff_count == 0:
        converged = True
    else:
        # Prepare for next iteration: rename new_component to component.
        vertices_df = updated_vertices.withColumnRenamed("new_component", "component")
        iteration += 1

# Show the Connected Components

print("Final vertices with computed connected component labels (using custom union-find):")
vertices_df.orderBy("component").show(20, truncate=False)


Iteration 0 number of component updates: 246
Iteration 1 number of component updates: 283
Iteration 2 number of component updates: 96
Iteration 3 number of component updates: 4
Iteration 4 number of component updates: 0
Final vertices with computed connected component labels (using custom union-find):
+---+---------+
|id |component|
+---+---------+
|BHM|ABE      |
|GSO|ABE      |
|MAF|ABE      |
|GSP|ABE      |
|ORD|ABE      |
|PVD|ABE      |
|GRR|ABE      |
|VPS|ABE      |
|CLE|ABE      |
|AMA|ABE      |
|RDU|ABE      |
|DEN|ABE      |
|IND|ABE      |
|DRO|ABE      |
|MHT|ABE      |
|KOA|ABE      |
|IAD|ABE      |
|RAP|ABE      |
|BDL|ABE      |
|SAT|ABE      |
+---+---------+
only showing top 20 rows



In [25]:
# Group vertices by component and count the number of vertices in each component.
components_group = vertices_df.groupBy("component").count().orderBy("count", ascending=False)
print("Connected component groups:")
components_group.show(truncate=False)

Connected component groups:
+---------+-----+
|component|count|
+---------+-----+
|ABE      |296  |
+---------+-----+

